In [1]:
import seaborn as sns
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, log_loss

In [2]:
param_array = pd.read_csv("parameters/parameter_array.csv").set_index("PARAM")

In [3]:
param_array

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
1,katl,1,PD,0,500,50,10
2,kclt,1,PD,0,500,50,10
3,kden,1,PD,0,500,50,10
4,kdfw,1,PD,0,500,50,10
5,kjfk,1,PD,0,500,50,10
...,...,...,...,...,...,...,...
716,kmem,12,PD_SM,1,500,50,10
717,kmia,12,PD_SM,1,500,50,10
718,kord,12,PD_SM,1,500,50,10


In [4]:
Param_Nums = []
DFs = []

Not_Finishes = []
for param in param_array.index:
    
    try:
        df = pd.read_csv(f"Results/Result_{param}/results_df.csv")
        Param_Nums.append(param)
        DFs.append(df)
        
        
    except:
        
        Not_Finishes.append(param)

    
df = pd.concat(DFs)
df["PARAM"] = Param_Nums
df = pd.merge(df, param_array, on="PARAM")

In [5]:
np.unique(df.LOOKAHEAD)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [6]:
df.columns

Index(['Unnamed: 0', 'param', 'train_loss', 'val_loss', 'test_loss',
       'train_accuracy', 'val_accuracy', 'test_accuracy', 'train_f1', 'val_f1',
       'test_f1', 'PARAM', 'AIRPORTS', 'LOOKAHEAD', 'DATA', 'CONFIG_SUPPORT',
       'EPOCHS', 'NUMBER_TRIALS', 'PATIENCE'],
      dtype='object')

In [7]:
df.groupby(["LOOKAHEAD","CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

LOOKAHEAD  CONFIG_SUPPORT  DATA
1          0               PD      0.028356
                           SM      0.031381
2          0               PD      0.046927
                           SM      0.069329
3          0               PD      0.074036
                           SM      0.082906
4          0               PD      0.062681
                           SM      0.075050
5          0               PD      0.079758
                           SM      0.077425
6          0               PD      0.069213
                           SM      0.083867
7          0               PD      0.074606
                           SM      0.101213
8          0               PD      0.097858
9          0               PD      0.113618
10         0               PD      0.094520
11         0               PD      0.103553
12         0               PD      0.063163
Name: test_loss, dtype: float64

In [8]:
df.groupby(["CONFIG_SUPPORT","DATA"]).mean()["test_loss"]

CONFIG_SUPPORT  DATA
0               PD      0.078441
                SM      0.078771
Name: test_loss, dtype: float64

In [9]:
param_array.loc[Not_Finishes]

,AIRPORTS,LOOKAHEAD,DATA,CONFIG_SUPPORT,EPOCHS,NUMBER_TRIALS,PATIENCE
PARAM,,,,,,,
4,kdfw,1,PD,0,500,50,10
5,kjfk,1,PD,0,500,50,10
6,kmem,1,PD,0,500,50,10
7,kmia,1,PD,0,500,50,10
8,kord,1,PD,0,500,50,10
...,...,...,...,...,...,...,...
716,kmem,12,PD_SM,1,500,50,10
717,kmia,12,PD_SM,1,500,50,10
718,kord,12,PD_SM,1,500,50,10
